## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,62.11,78,0,3.44,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,46.11,93,99,9.13,overcast clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,65.17,93,49,8.52,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,69.48,50,0,3.22,clear sky
4,4,Leningradskiy,RU,69.3833,178.4167,1.29,86,8,1.43,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Dwarka,IN,22.2394,68.9678,82.63,58,6,8.77,clear sky
8,8,Salalah,OM,17.0151,54.0924,82.49,37,100,1.50,overcast clouds
12,12,Avarua,CK,-21.2078,-159.7750,80.65,83,75,8.05,broken clouds
17,17,Rikitea,PF,-23.1203,-134.9692,79.39,71,2,13.44,clear sky
18,18,Hithadhoo,MV,-0.6000,73.0833,82.31,70,12,9.78,few clouds
21,21,Cockburn Town,TC,21.4612,-71.1419,77.54,71,37,3.36,scattered clouds
22,22,New Norfolk,AU,-42.7826,147.0587,80.10,36,0,3.00,clear sky
28,28,Mackay,AU,-21.1500,149.2000,82.38,69,40,12.66,scattered clouds
30,30,Kapaa,US,22.0752,-159.3190,78.78,70,3,1.01,clear sky
32,32,Sorong,ID,-0.8833,131.2500,82.76,75,91,5.30,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Dwarka,IN,82.63,clear sky,22.2394,68.9678,
8,Salalah,OM,82.49,overcast clouds,17.0151,54.0924,
12,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,
17,Rikitea,PF,79.39,clear sky,-23.1203,-134.9692,
18,Hithadhoo,MV,82.31,few clouds,-0.6000,73.0833,
21,Cockburn Town,TC,77.54,scattered clouds,21.4612,-71.1419,
22,New Norfolk,AU,80.10,clear sky,-42.7826,147.0587,
28,Mackay,AU,82.38,scattered clouds,-21.1500,149.2000,
30,Kapaa,US,78.78,clear sky,22.0752,-159.3190,
32,Sorong,ID,82.76,overcast clouds,-0.8833,131.2500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
hotel_df.head(10)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Dwarka,IN,82.63,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
8,Salalah,OM,82.49,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
12,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,Paradise Inn
17,Rikitea,PF,79.39,clear sky,-23.1203,-134.9692,People ThankYou
18,Hithadhoo,MV,82.31,few clouds,-0.6000,73.0833,Scoop Guest House
21,Cockburn Town,TC,77.54,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
22,New Norfolk,AU,80.10,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
28,Mackay,AU,82.38,scattered clouds,-21.1500,149.2000,International Lodge Motel
30,Kapaa,US,78.78,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
32,Sorong,ID,82.76,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel


In [9]:
hotel_df.tail(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
622,Moindou,NC,79.57,broken clouds,-21.6924,165.6770,Equi-lodge de Moindou
633,Byron Bay,AU,76.71,overcast clouds,-28.6500,153.6167,Backpackers Inn
634,Baracoa,CU,73.31,overcast clouds,20.3467,-74.4958,Hostal Rio Miel
635,Jizan,SA,76.14,scattered clouds,17.3333,42.6667,شهدان. ابوالحجر
636,Nioro,GM,71.51,broken clouds,13.3500,-15.7500,
641,Kabalo,CD,70.23,overcast clouds,-6.0500,26.9167,
647,Saint-Marc,HT,76.53,broken clouds,19.1082,-72.6938,Maguana Hotel
648,Mbaiki,CF,70.45,broken clouds,3.8678,17.9892,Motel MBM
651,Sabya,SA,76.62,scattered clouds,17.1495,42.6254,Aseel Hotel Apartment
654,San Juan Del Sur,NI,78.26,scattered clouds,11.2529,-85.8705,Nuestra Casa


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.tail()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
635,Jizan,SA,76.14,scattered clouds,17.3333,42.6667,شهدان. ابوالحجر
647,Saint-Marc,HT,76.53,broken clouds,19.1082,-72.6938,Maguana Hotel
648,Mbaiki,CF,70.45,broken clouds,3.8678,17.9892,Motel MBM
651,Sabya,SA,76.62,scattered clouds,17.1495,42.6254,Aseel Hotel Apartment
654,San Juan Del Sur,NI,78.26,scattered clouds,11.2529,-85.8705,Nuestra Casa


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")



In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))